In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as T
from torch.utils.data import DataLoader
from torch.profiler import profile, record_function, ProfilerActivity

In [2]:
def get_dataloaders(batch_size=128, num_workers=4):
    """
    TODO:
      - Compose transforms (Resize→32, ToTensor, Normalize)
      - Load CIFAR10 train & test datasets with torchvision.datasets.CIFAR10
      - Return train_loader and test_loader DataLoader objects
    """
    transform = T.Compose([
        T.Resize(32),
        T.ToTensor(),
        T.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
    ])
    train_ds = torchvision.datasets.CIFAR10(
        root='./data', train=True, download=True, transform=transform
    )
    test_ds = torchvision.datasets.CIFAR10(
        root='./data', train=False, download=True, transform=transform
    )
    train_loader = DataLoader(train_ds, batch_size=batch_size,
                              shuffle=True, num_workers=num_workers)
    test_loader  = DataLoader(test_ds,  batch_size=batch_size,
                              shuffle=False, num_workers=num_workers)
    return train_loader, test_loader

In [4]:
def build_model(device):
    """
    TODO:
      - Instantiate torchvision.models.resnet18(pretrained=False)
      - Replace final fc layer with nn.Linear(in_features, 10)
      - Move model to `device` and return it
    """
    model = torchvision.models.resnet18(pretrained=False)
    model.fc = nn.Linear(model.fc.in_features, 10)
    return model.to(device)


In [5]:
def train_step(model, data, target, optimizer, criterion):
    """
    TODO:
      - Zero grads, do forward(model(data)), compute loss,
        backward(), step optimizer
      - Return the loss value (item)
    """
    optimizer.zero_grad()
    outputs = model(data)
    loss = criterion(outputs, target)
    loss.backward()
    optimizer.step()
    return loss.item()

In [6]:
def inference_step(model, data):
    """
    TODO:
      - Run a forward pass under torch.no_grad()
    """
    with torch.no_grad():
        _ = model(data)

In [7]:
def start():
    # 1) Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 2) Load data
    train_loader, test_loader = get_dataloaders(batch_size=128, num_workers=4)

    # 3) Build model + optimizer + loss
    model     = build_model(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    criterion = nn.CrossEntropyLoss()

    # 4) Warm up on one batch
    data, target = next(iter(train_loader))
    data, target = data.to(device), target.to(device)
    train_step(model, data, target, optimizer, criterion)
    inference_step(model, data)

    # 5) Prepare profiler logdir
    logdir = "./profiler_logs"
    os.makedirs(logdir, exist_ok=True)

    # 6) Profile exactly one train + one inference
    with profile(
        activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
        record_shapes=True,
        profile_memory=True,
        with_stack=True,
        on_trace_ready=torch.profiler.tensorboard_trace_handler(logdir)
    ) as prof:
        with record_function("train_batch"):
            train_step(model, data, target, optimizer, criterion)
        with record_function("inference_batch"):
            inference_step(model, data)

    # 7) Print top‐3 CPU ops
    print("\n=== Top 3 ops by CPU self time ===")
    print(prof.key_averages().table(
        sort_by="self_cpu_time_total", row_limit=3
    ))

    # 8) Print top‐3 CUDA ops
    print("\n=== Top 3 ops by CUDA self time ===")
    print(prof.key_averages().table(
        sort_by="self_cuda_time_total", row_limit=3
    ))

    print(f"\nTrace files written to: {logdir}")
    print("Run `tensorboard --logdir profiler_logs` and open the Profile dashboard.")

In [8]:
start()

/opt/conda/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



=== Top 3 ops by CPU self time ===
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                       Runtime Triggered Module Loading        31.44%      16.295ms        31.44%      16.295ms       8.148ms     761.836us         1.87%     761.836us     380.918us           0 b     